### 矩阵的 sum 运算

In [298]:
import numpy as np
import torch
from IPython import display
from d2l import torch as d2l

X = np.array([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
# 0 ==> 行方向 ==> 压缩成一行
# 1 ==> 列方向 ==> 压缩成一列
X.sum(0, keepdims=True), X.sum(1, keepdims=True)

(array([[5., 7., 9.]]),
 array([[ 6.],
        [15.]]))

In [299]:
X = torch.tensor([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
X.sum(0, keepdim=True), X.sum(1, keepdim=True)

(tensor([[5., 7., 9.]]),
 tensor([[ 6.],
         [15.]]))

### 加载数据集

In [300]:
batch_size = 256 # 指明每一次数据的迭代都会抛出 batch_size 个样本数据
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size)

### 定义 Softmax 函数

$$
\operatorname{softmax}(\mathbf{X})_{i j}=\frac{\exp \left(\mathbf{X}_{i j}\right)}{\sum_{k} \exp \left(\mathbf{X}_{i k}\right)}
$$

In [301]:
def softmax(X):
    """
    softmax 函数的输入是一个矩阵
    这里的 X 是经过神经网络的线性变换后的输出
    而不是单纯的样本数据；
    softmax 回归是一个分类器，它有多个输出端
    X 矩阵的每一行都是一个样本数据经过神经网络后的分类个数个的输出值
    之所以是一个矩阵，这是为了兼容一次性输入多个样本后的多个样本的输出
    """
    X_exp = np.exp(X)
    partition = X_exp.sum(1, keepdims=True)
    return X_exp / partition # 这里用到了广播，partition 横向扩展成矩阵

In [302]:
X = np.array([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
softmax(X), softmax(X).sum(1, keepdims=True)

(array([[0.09003057, 0.24472847, 0.66524096],
        [0.09003057, 0.24472847, 0.66524096]]),
 array([[1.],
        [1.]]))

### 初始化参数

In [303]:
num_inputs = 784
num_outputs = 10

W = torch.normal(0, 0.01, size=(num_inputs, num_outputs), requires_grad=True)
b = torch.zeros(num_outputs, requires_grad=True)

### 定义网络模型

In [304]:
def net(X):
    """
    X.reshape 和 W 点乘后记为矩阵 C
    C 的每一行都是一个样本数据经过 num_outpus 个神经元线性变化但没有加偏置项的输出
    所以偏置项应该依次加在 C 矩阵每一行上的每一个元素，这样：b 是一个行向量是没有问题的
    广播机制后，复制 行向量b 纵向扩展为一个矩阵
    """
    return softmax(torch.matmul(X.reshape((-1, W.shape[0])), W) + b)

In [305]:
y = torch.tensor([0, 2])
y_hat = torch.tensor([[0.1, 0.3, 0.6], [0.3, 0.2, 0.5]])
# A[[a, b], [c, d]] 
# ==> 选取了 A 矩阵的 ab 位置的元素 和 cd 位置的元素
# Aab 和 Acd 两个元素 

# 选取 0 号样本的  0 号类别的预测概率
# 选取 1 号样本的 2 号类别的预测概率
y_hat[[0, 1], y]

tensor([0.1000, 0.5000])

### 定义损失函数

In [306]:
def cross_entropy(y_hat, y):
    """
    y_hat 是一个矩阵，该矩阵的第 i 号行为 i 号样本的预测概率分布向量
    y_prob 是 label 的 one-hot 编码
    y_prob[i] = [0, 0, ... , 1, 0, 0] 只有第 j 号是 1，代表它是第 j 号类别
    对于样本 i 而言，-log(y_hat[i])*y_prob[i]
    由于 y_prob[i] 向量中只有第 j 号元素是 1，其他元素都为 0
    那么 y_prob[i] 作为系数而言为 1 的那一项的系数可以不写，而其他所有的项的系数都为 0
    所以 i 号样本的交叉熵计算过程中，其实只有一项被保留下来了，其他项都由于 0 这个系数被消除了
    则，属于 j 号类别的 i 号样本的交叉熵为 : -log(y_hat[i][j])*y_prob[i][j]=-log(y_hat[i][j])
    """
    return - torch.log(y_hat[range(len(y_hat)), y])

# 得到的返回值是一个向量 ==> 所有样本的交叉熵损失值
cross_entropy(y_hat, y)

tensor([2.3026, 0.6931])

### argmax 函数使用

In [307]:
X = torch.tensor([
    [1, 2, 3],
    [6, 5, 4],
    [7, 9, 8]
])

In [308]:
# 在列方向上压缩成一列
# 但是返回的 res 的值
# 并不是 X 矩阵中的元素
# 而是 X 矩阵中每一列的最大值的下标
res = X.argmax(axis=1, keepdim=True)
res

tensor([[2],
        [0],
        [1]])

### 分类准确度函数

In [309]:
def accuracy(y_hat, y):  #@save
    """计算预测正确的数量"""
    if len(y_hat.shape) > 1 and y_hat.shape[1] > 1:
        # 把概率分布 y_hat 压缩为与 y 类似的标签
        # y_hat 中 i 行上的向量中最大的概率的下标
        # 就是样本 i 的预测类别的离散数值表示
        y_hat = y_hat.argmax(axis=1)
    cmp = y_hat.type(y.dtype) == y
    return float(cmp.type(y.dtype).sum())

In [310]:
# 举个例子 ==> 前边已经定义过 y_hat 和 y 了
# y = torch.tensor([1, 0])
# y_hat = softmax(torch.tensor([
#     [6, 9],
#     [16, 10]
# ]).reshape(2, 2))
# y, y_hat

In [311]:
label_hat = y_hat.argmax(axis=1, keepdim=True)
label_hat

tensor([[2],
        [2]])

In [312]:
# 使用 accuracy 函数试验
# accuracy 计算的是预测正确的数量而不是比例
accuracy(y_hat, y), accuracy(y_hat, y) / len(y)

(1.0, 0.5)

### 定义 Accumulator

In [313]:
# 使用 Accumulator 是因为
# 在预测数据集过大的时候，不能够一次性加入内存的时候
# 预测正确的数量 不能够一次性统计完成
# 所以需要每预测以小批量的测试样本后就统计一次预测正确的数值类加上去
# 最后完成了所有测试数据集的样本后，
# 得到 ==> 分类准确度 = 预测正确的数量 / 总数

class Accumulator:  #@save
    """在n个变量上累加"""
    def __init__(self, n):
        self.data = [0.0] * n

    def add(self, *args):
        self.data = [a + float(b) for a, b in zip(self.data, args)]

    def reset(self):
        self.data = [0.0] * len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

In [314]:
def evaluate_accuracy(net, data_iter):  #@save
    """计算在指定数据集上模型的精度"""
    # 应该是对于 Moudle 的类型需要打开评估模式
    # 才能够这样使用么 ~ 疑问句 ~  
    if isinstance(net, torch.nn.Module):
        net.eval()  # 将模型设置为评估模式
    metric = Accumulator(2)  # 正确预测数、预测总数
    with torch.no_grad():
        for X, y in data_iter:
            # numel ==> number of elements 元素个数
            metric.add(accuracy(net(X), y), y.numel())
    return metric[0] / metric[1]

In [316]:
evaluate_accuracy(net, test_iter), len(next(iter(test_iter))[0])

(0.1127, 256)